In [80]:
%matplotlib inline

import math
import keras
import keras.backend as K
import datetime
import matplotlib.pyplot as plt
import xgboost
import operator
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas_summary import DataFrameSummary
from isoweek import Week
pd.options.display.max_columns = 50

In [81]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_stores = pd.read_csv('store.csv')
df_train_full = df_train.merge(df_stores, on='Store')
df_test_full = df_test.merge(df_stores, on='Store')
df_all_full = pd.concat([df_train_full, df_test_full])

In [82]:
df_train_full.columns

Index([u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open',
       u'Promo', u'StateHoliday', u'SchoolHoliday', u'StoreType',
       u'Assortment', u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'PromoInterval'],
      dtype='object')

In [83]:
df_test_full.columns

Index([u'Id', u'Store', u'DayOfWeek', u'Date', u'Open', u'Promo',
       u'StateHoliday', u'SchoolHoliday', u'StoreType', u'Assortment',
       u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'PromoInterval'],
      dtype='object')

In [70]:
df_train_full[(df_train_full.Store == 1) & (df_train_full.DayOfWeek == 7)]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
5,1,7,2015-07-26,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
12,1,7,2015-07-19,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
19,1,7,2015-07-12,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
26,1,7,2015-07-05,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
33,1,7,2015-06-28,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
40,1,7,2015-06-21,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
47,1,7,2015-06-14,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
54,1,7,2015-06-07,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
61,1,7,2015-05-31,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
68,1,7,2015-05-24,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [23]:
df_test_full.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,857,1,3,2015-09-16,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1713,1,2,2015-09-15,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,2569,1,1,2015-09-14,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,3425,1,7,2015-09-13,0.0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [24]:
DataFrameSummary(df_train_full).summary()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,1.01721e+06,NaN,NaN,1.01457e+06,693861,693861,1.01721e+06,509178,509178,NaN
mean,558.43,3.99834,NaN,5773.82,633.146,0.830107,0.381515,NaN,0.178647,NaN,NaN,5430.09,7.22287,2008.69,0.500564,23.2691,2011.75,NaN
std,321.909,1.99739,NaN,3849.93,464.412,0.375539,0.485759,NaN,0.383056,NaN,NaN,7715.32,3.21183,5.99264,0.5,14.096,1.66287,NaN
min,1,1,NaN,0,0,0,0,NaN,0,NaN,NaN,20,1,1900,0,1,2009,NaN
25%,280,2,NaN,3727,405,1,0,NaN,0,NaN,NaN,710,4,2006,0,13,2011,NaN
50%,558,4,NaN,5744,609,1,0,NaN,0,NaN,NaN,2330,8,2010,1,22,2012,NaN
75%,838,6,NaN,7856,837,1,1,NaN,0,NaN,NaN,6890,10,2013,1,37,2013,NaN
max,1115,7,NaN,41551,7388,1,1,NaN,1,NaN,NaN,75860,12,2015,1,50,2015,NaN
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1014567,693861,693861,1017209,509178,509178,509178
uniques,1115,7,942,21734,4086,2,2,5,2,4,3,654,12,23,2,24,7,3


In [26]:
def prepare_dataset(df, one_hot = False):
    df.Date = pd.to_datetime(df.Date)
    df["Year"] = df.Date.dt.year
    df["Month"] = df.Date.dt.month
    df["Week"] = df.Date.dt.week
    df["Day"] = df.Date.dt.day
    df['StateHoliday'] = df['StateHoliday'] != '0'
    df['SchoolHoliday'] = df['SchoolHoliday'] != 0
    df['CompetitionDistance'] = df['CompetitionDistance'].fillna(df['CompetitionDistance'].mean())
    df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].fillna(1).astype(np.int32)
    df["CompetitionOpenSinceDateTime"] = pd.to_datetime(df.apply(lambda x: datetime.datetime(
    x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 1), axis=1).astype(pd.datetime))
    df["CompetitionDaysElapsed"] = df.Date.subtract(df["CompetitionOpenSinceDateTime"]).dt.days
    df['Promo2SinceYear'] = df['Promo2SinceYear'].fillna(2009).astype(np.int32)
    df['Promo2SinceWeek'] = df['Promo2SinceWeek'].fillna(1).astype(np.int32)
    df["Promo2SinceDateTime"] = pd.to_datetime(df.apply(lambda x: Week(
    x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2SinceDaysElapsed"] = df.Date.subtract(df["Promo2SinceDateTime"]).dt.days
    return df

In [ ]:
def create_store_dictionary(df):
    df = df.sort_values(['Date'])
    store_dict = {}
    for store in np.unique(df.Store):
        df_store = df[df.Store == store]
        sales_mean = df.loc[(df.Sales > 0.0) & (df.Store == store),"Sales"].mean()
        costumer_mean = df.loc[(df.Customers > 0.0) & (df.Store == store), "Customers"].mean()
        is_open_sunday = df_train_full.loc[(df_train_full.Sales > 0.0) & (df_train_full.DayOfWeek == 7) & (df_train_full.Store == store),"Customers"].empty
        state_hols_array = df_store.StateHoliday.values
        state_hols_starting_date = df_store.Date[np.append([state_hols_array[0] == 1], (state_hols_array[:-1] == 0) & (state_hols_array[1:] == 1))].values
        school_hols_array = df_store